In [ ]:
%config InlineBackend.figure_format='retina'

In [ ]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scanpy as sc

from cytofuture_data.gene_name_mapping import GeneNameMapper

In [ ]:
# Load the gene name mapper
gene_name_mapper = GeneNameMapper(
    '../standard_genes/gene_names/human_genes.csv',
    '../standard_genes/gene_names/mouse_genes.csv',
    '../standard_genes/gene_names/orthologue_map_human2mouse_best.csv',
    '../standard_genes/gene_names/orthologue_map_mouse2human_best.csv'
)

In [ ]:
# Load the data
data_collection_name = 'Qiu_Organogenesis_MM_2022'
data_path = os.path.join('/home/xingjie/Data/data2/cytofuture/datasets/scRNAseq/', data_collection_name)
dataset_name = 'all'
query_organism, query_var_id_type = 'mouse', 'id'

adata = sc.read_h5ad(os.path.join(data_path, 'TOME', 'adata_all.h5ad'))
adata

In [ ]:
sc.pp.calculate_qc_metrics(adata, percent_top=None, log1p=False, inplace=True)
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts'], jitter=0, multi_panel=True, log=True)

In [ ]:
sc.pp.filter_cells(adata, min_genes=200)

In [ ]:
# Append the dataset name into cel IDs
adata.obs.index = [':'.join([data_collection_name, dataset_name, i]) for i in adata.obs.index]

# Copy the standard meta-data
adata_std = sc.AnnData(X=adata.X, obs=adata.obs[[]], var=adata.var[[]], dtype=np.float32)
adata_std.obs['dataset_id'] = ':'.join([data_collection_name, dataset_name]) 
adata_std.obs['assay'] = 'seq'
adata_std.obs['batch'] = [':'.join([data_collection_name, dataset_name, a]) 
                         for a in adata.obs['orig.ident']]
adata_std.obs['development_stage'] = adata.obs['day']
adata_std.obs['tissue'] = adata.obs['cell_type']
adata_std.obs['cell_type'] = adata.obs['cell_type']

# Map the genes to human gene IDs
adata_std.var['human_gene_id'] = gene_name_mapper.map_gene_names(list(adata_std.var.index), 
                                                             query_organism, 'human', query_var_id_type, 'id')

In [ ]:
print(np.unique(adata_std.obs['cell_type']))
print(np.unique(adata_std.obs['development_stage']))
display(adata_std)
display(adata_std[:, adata_std.var['human_gene_id'] != 'na'])

In [ ]:
%%time
adata_std.write_h5ad(os.path.join(data_path, f'standard_adata_{data_collection_name}_{dataset_name}_origional_cell_type.h5ad'),
                compression='gzip')

In [ ]:
data_path